In [2]:
!pip install torchvision

   ---------------------------------------- 0.0/3.7 MB ? eta -:--:--
   ----------------- ---------------------- 1.6/3.7 MB 21.1 MB/s eta 0:00:01
   ---------------------------------------- 3.7/3.7 MB 20.1 MB/s eta 0:00:00


In [3]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [4]:
class HairCNN(nn.Module):
    def __init__(self):
        super(HairCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=0, stride=1)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.relu2 = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu2(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

model = HairCNN().to(device)
print(model)

HairCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (relu2): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


## Question 1: Loss Function
For binary classification with sigmoid output: `nn.BCELoss()` or `nn.BCEWithLogitsLoss()` (if no sigmoid in model)

In [5]:
criterion = nn.BCELoss()

## Question 2: Total Parameters

In [6]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


In [7]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = ImageFolder('data/train', transform=train_transforms)
validation_dataset = ImageFolder('data/test', transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(validation_dataset)}")
print(f"Classes: {train_dataset.classes}")

Train samples: 800
Test samples: 201
Classes: ['curly', 'straight']


In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [9]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (outputs > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (outputs > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6462, Acc: 0.6362, Val Loss: 0.6032, Val Acc: 0.6517
Epoch 2/10, Loss: 0.5475, Acc: 0.7100, Val Loss: 0.7251, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5533, Acc: 0.7250, Val Loss: 0.5991, Val Acc: 0.6716
Epoch 4/10, Loss: 0.4802, Acc: 0.7712, Val Loss: 0.6033, Val Acc: 0.6567
Epoch 5/10, Loss: 0.4334, Acc: 0.8025, Val Loss: 0.6196, Val Acc: 0.6766
Epoch 6/10, Loss: 0.3740, Acc: 0.8325, Val Loss: 0.7371, Val Acc: 0.6766
Epoch 7/10, Loss: 0.2721, Acc: 0.8838, Val Loss: 0.9223, Val Acc: 0.6418
Epoch 8/10, Loss: 0.2478, Acc: 0.9000, Val Loss: 0.7294, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2075, Acc: 0.9200, Val Loss: 0.7523, Val Acc: 0.7015
Epoch 10/10, Loss: 0.1462, Acc: 0.9463, Val Loss: 0.7970, Val Acc: 0.6915


## Question 3: Median of Training Accuracy

In [10]:
median_train_acc = np.median(history['acc'])
print(f"Median of training accuracy: {median_train_acc:.4f}")

Median of training accuracy: 0.8175


## Question 4: Standard Deviation of Training Loss

In [11]:
std_train_loss = np.std(history['loss'])
print(f"Standard deviation of training loss: {std_train_loss:.4f}")

Standard deviation of training loss: 0.1594


## Data Augmentation Training (Questions 5 & 6)

In [12]:
train_transforms_aug = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset_aug = ImageFolder('data/train', transform=train_transforms_aug)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)

In [14]:
history_aug = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (outputs > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train
    history_aug['loss'].append(epoch_loss)
    history_aug['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (outputs > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history_aug['val_loss'].append(val_epoch_loss)
    history_aug['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.7383, Acc: 0.6100, Val Loss: 0.5859, Val Acc: 0.6866
Epoch 2/10, Loss: 0.5736, Acc: 0.6550, Val Loss: 0.5606, Val Acc: 0.7114
Epoch 3/10, Loss: 0.5785, Acc: 0.6575, Val Loss: 0.5220, Val Acc: 0.7164
Epoch 4/10, Loss: 0.5713, Acc: 0.6950, Val Loss: 0.5686, Val Acc: 0.7114
Epoch 5/10, Loss: 0.5291, Acc: 0.7150, Val Loss: 0.5687, Val Acc: 0.6667
Epoch 6/10, Loss: 0.5301, Acc: 0.7075, Val Loss: 0.5935, Val Acc: 0.6567
Epoch 7/10, Loss: 0.5178, Acc: 0.7238, Val Loss: 0.4946, Val Acc: 0.7413
Epoch 8/10, Loss: 0.5306, Acc: 0.7238, Val Loss: 0.6094, Val Acc: 0.6517
Epoch 9/10, Loss: 0.5122, Acc: 0.7288, Val Loss: 0.5934, Val Acc: 0.6667
Epoch 10/10, Loss: 0.5016, Acc: 0.7400, Val Loss: 0.5346, Val Acc: 0.7363


## Question 5: Mean of Test Loss (with augmentation)

In [15]:
mean_val_loss = np.mean(history_aug['val_loss'])
print(f"Mean of test loss: {mean_val_loss:.4f}")

Mean of test loss: 0.5631


## Question 6: Average Test Accuracy (last 5 epochs)

In [16]:
avg_val_acc_last5 = np.mean(history_aug['val_acc'][5:])
print(f"Average test accuracy (epochs 6-10): {avg_val_acc_last5:.4f}")

Average test accuracy (epochs 6-10): 0.6905
